In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nltk

# Let's prepare data
nltk.download('names')
from nltk.corpus import names

male_names = names.words('male.txt')
female_names = names.words('female.txt')

# Let's merge the datasets and create labels
data = [(name.lower(), 0) for name in male_names] + [(name.lower(), 1) for name in female_names]

np.random.shuffle(data) # Shuffle the data

# Tokenization and Encoding
all_letters = "abcdefghijklmnopqrstuvwxyz"
n_letters = len(all_letters)

def name_to_tensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for i, letter in enumerate(name):
        tensor[i][0][all_letters.find(letter)] = 1
    return tensor

# Model Architecture
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        lstm_out, _ = self.lstm(input)
        output = self.fc(lstm_out[-1])
        return output

# Training
model = LSTMClassifier(n_letters, 128, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(data):
    for name, label in data:
        input_tensor = name_to_tensor(name)
        target = torch.tensor([label], dtype=torch.long)
        
        optimizer.zero_grad()
        
        output = model(input_tensor)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Training loop
for epoch in range(5):
    train(data)
    print(f"Epoch {epoch+1} completed.")

# Inference
def predict(name, model):
    with torch.no_grad():
        input_tensor = name_to_tensor(name.lower())
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        if predicted.item() == 0:
            return "Male"
        else:
            return "Female"

# Let's pick a random English name as an example usage
name_to_classify = "Jean"
print(f"The name {name_to_classify} is classified as {predict(name_to_classify, model)}")

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\flore\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
The name Jean is classified as Female
